In [39]:
import pandas as pd
import gurobipy as gp
import math

In [49]:
article_data = pd.read_csv("./data/article.csv")
article_data['MAXIMUM_ORDER_QUANTITY_TU'].fillna(100000, inplace=True)

article_data = article_data[(article_data['TEMPERATURE_ZONE'] == 'frozen')]

In [41]:
# constants definitions
warehouse_volume = 50
buffer_cost = 25

In [42]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [48]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [44]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu,minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [45]:
time_periods = ['2022-06-13','2022-06-14','2022-06-15','2022-06-16','2022-06-17','2022-06-18']
forecast_data = pd.read_csv('./data/sales_7.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]
demand = dict.fromkeys(articles)
for item in demand.keys():
    demand[item] = dict.fromkeys(time_periods)
    for time_period in demand[item].keys():
        count = forecast_data.loc[(forecast_data['ARTICLE_ID'] == item) & (forecast_data['DATE'] == time_period)]['PICKING_QUANTITY_CU']
        if(len(count)==0):
            demand[item][time_period] = 0
        else:
            demand[item][time_period] = count.values[0]

In [47]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_periods, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_periods, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, time_period] for item in items for time_period in time_periods)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[time_period] for time_period in time_periods)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in items:
    for time_period_iterator, time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr(orders[item, time_period] * cu_per_tu[item] >= demand[item][time_period],name="demand constraint_" + str(time_period))
        else:
            m.addConstr(orders[item, time_period] * cu_per_tu[item] + storage_used[item, time_periods[time_period_iterator-1]]>= demand[item][time_period],name="demand constraint_" + str(time_period))

# inventory volume constraint
for time_period_iterator, time_period in enumerate(time_periods):
    if(time_period_iterator==0):
        m.addConstr(gp.quicksum(volume_per_cu[item] * cu_per_tu[item] * (orders[item, time_period]) for item in items) <= warehouse_volume + buffer_storage_used[time_period])
    else:
        m.addConstr(gp.quicksum(volume_per_cu[item] * (storage_used[item, time_periods[time_period_iterator-1]] + (cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])

# min/max constraints (linking too)
for item in items:
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= minimum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        m.addConstr(orders[item, time_period] <= maximum_order_quantity_tu[item] * ordered_boolean[item, time_period])

m.setParam(gp.GRB.Param.ScaleFlag, 2)  
m.setParam(gp.GRB.Param.NumericFocus, 1)     
m.optimize()

Set parameter ScaleFlag to value 2
Set parameter NumericFocus to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 119976 rows, 119976 columns and 271784 nonzeros
Model fingerprint: 0x6835d4b0
Variable types: 0 continuous, 119976 integer (39990 binary)
Coefficient statistics:
  Matrix range     [2e-08, 1e+05]
  Objective range  [1e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 53651 rows and 6665 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.06 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
